In [1]:
import os
if os.path.basename(os.getcwd())!="HUST-CV-Neural-Style-Transfer":
    %cd ../../

e:\pyenv\GTCC\KPG-RL\HUST-CV-Neural-Style-Transfer


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import torchvision.transforms.v2 as transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import math
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import pandas as pd
from sklearn.model_selection import train_test_split

c:\Users\tungv\AppData\Local\Programs\Python\Python311\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.6 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.resnet = nn.Sequential(*list(models.resnet34(pretrained=True).children())[:-1])        
        for param in self.resnet.parameters():
            param.requires_grad = False
        self.resnet.eval()
        
        self.flat = nn.Flatten()
        
        self.linear = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.resnet(x)
        x = self.flat(x)
        x = self.linear(x)
        
        return x

In [5]:
#load model
model = Model().to(DEVICE)
model.load_state_dict(torch.load("models/model_rating.pth"))

c:\Users\tungv\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tungv\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\tungv\AppData\Local\Temp\ipykernel_3908\4284162177.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickli

<All keys matched successfully>

In [7]:
transform = transforms.Compose([
            transforms.Resize([456,456]),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize the image
        ])

def predict(model, image_path):
    image = Image.open(image_path)
    image = transform(image)
    output = model(image.unsqueeze(0).to(DEVICE))
    output = output.cpu().detach().numpy()
    return output[0][0] * 10

predict(model,"outputs/final_img_0_0.png")

5.521048903465271